In [ ]:
from pathlib import Path
import os.path



In [ ]:
from livecellx.track.classify_utils import load_class2samples_from_json_dir, load_all_json_dirs


sample_json_dirs_v0 = [Path(r"../datasets/test_scs_EBSS_starvation/XY1/annotations"), Path(r"./datasets/test_scs_EBSS_starvation/XY16/annotations")]
round1_json_dirs = sample_json_dirs_v0 + [
    Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY01\XY01"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY09\XY09"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\shiman_XY10\XY10"),
Path(r"D:\LiveCellTracker-dev\datasets\mitosis-annotations-2023\Yajushi\tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19\XY1\annotations"),
]

round2_json_dirs = [
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.5^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C0.75^4/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^3/"),
Path(r"../datasets/mitosis-annotations-2023/shiman_CXA_high_density/C10^4/")
] + [
    Path(f"../datasets/mitosis-annotations-2023/Gaohan_tifs_CFP_A549-VIM_lessThan24hr_NoTreat_NA_YL_Ti2e_2022-10-19/XY{pos}/annotations") for pos in range(4, 14)
]

sample_json_dirs = sample_json_dirs_v0 + round1_json_dirs + round2_json_dirs
all_class2samples, all_class2sample_extra_info = load_all_json_dirs(sample_json_dirs)

In [ ]:
for key in all_class2samples:
    print(key, len(all_class2samples[key]))

In [ ]:
mitosis_sampels = all_class2samples["mitosis"]
mitosis_sampels[0][0].meta

Store sample index in all_class2samples to each single cell for future use

In [ ]:
for idx, sample in enumerate(mitosis_sampels):
    if len(sample) == 0:
        continue
    _first_time = sample[0].timeframe
    for sc in sample:
        sc.meta["_sample_idx"] = idx
        sc.meta["mitosis_relative_time"] = sc.timeframe - _first_time


In [ ]:
all_scs = [sc for sample in mitosis_sampels for sc in sample]

In [ ]:
from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize
inputs = []
for sc in all_scs:
    # features = compute_skimage_regionprops(sc, preprocess_img_func=normalize_img_to_uint8, sc_level_normalize=True)
    inputs.append({
        "sc": sc,
        "feature_key": "skimage",
        "preprocess_img_func": normalize_img_to_uint8,
        "sc_level_normalize": True,
    })

def compute_skimage_regionprops_wrapper(**input):
    sc = input["sc"]
    compute_skimage_regionprops(**input)
    return sc

# TODO: debug
# processed_scs = parallelize(compute_skimage_regionprops_wrapper, inputs)

# a for loop for computing
import tqdm
processed_scs = []
for input in tqdm.tqdm(inputs):
    sc = input["sc"]
    compute_skimage_regionprops(**input)
    processed_scs.append(sc)

In [ ]:
def create_sc_table(scs, normalize_features=True, add_time=True, add_sc_id=False):
    import pandas as pd
    import numpy as np
    df = pd.DataFrame([sc.get_feature_pd_series() for sc in scs])
    if normalize_features:
        for col in df.columns:
            df[col] = (df[col] - df[col].mean())
            col_std = df[col].std()
            if col_std != 0 and not np.isnan(col_std):
                df[col] /= col_std
    # remove column t from df
    if not add_time:
        df.drop("t", axis=1, inplace=True)
    if add_sc_id:
        df["sc_id"] = [sc.id for sc in scs]
    return df
sc_table_with_time = create_sc_table(processed_scs, add_time=False)
sc_feature_table = create_sc_table(processed_scs, add_time=True)

In [ ]:
# save the tables
out_dir = Path("./figure_mitosis_trajs_viz")
out_dir.mkdir(exist_ok=True)
sc_table_with_time.to_csv(out_dir / "mitosis_scs_table_with_time.csv")

In [ ]:
sc_table_with_time[:2]

In [ ]:
!pip install umap-learn

In [ ]:
import umap
reducer = umap.UMAP()
# drop NAN in sc_feature_table
sc_feature_table = sc_feature_table.dropna(axis=1)

embedding = reducer.fit_transform(sc_feature_table)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sc.timeframe for sc in processed_scs],
    alpha=0.5,
)
plt.title("Ignore this figure: meaningless timeframe")
plt.xlabel("UMAP_1")
plt.ylabel("UMAP_2")
# add colorbar
plt.colorbar()

In [ ]:
mitosis_colors = [sc.meta["mitosis_relative_time"] for sc in processed_scs]
mitosis_colors = [min(10, x) for x in mitosis_colors]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=mitosis_colors,
    cmap="inferno",
    alpha=0.5,
)
plt.xlabel("UMAP_1")
plt.ylabel("UMAP_2")
plt.title("Mitosis: relative time")
# add colorbar
plt.colorbar()

In [ ]:
sc_feature_table.columns

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=sc_feature_table["skimage_area"],
    cmap="inferno",
    alpha=0.5,
)
axes[0].set_xlabel("UMAP_1")
axes[0].set_ylabel("UMAP_2")
axes[0].set_title("Mitosis: area")
# add colorbar
# axes[0].colorbar()

axes[1].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=mitosis_colors,
    cmap="inferno",
    alpha=0.5,
)
axes[1].set_xlabel("UMAP_1")
axes[1].set_ylabel("UMAP_2")
axes[1].set_title("Mitosis: relative time")

axes[2].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=sc_feature_table["skimage_eccentricity"],
    cmap="inferno",
    alpha=0.5,
)
axes[2].set_xlabel("UMAP_1")
axes[2].set_ylabel("UMAP_2")
axes[2].set_title("Mitosis: eccentricity")


axes[3].scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=sc_feature_table["skimage_orientation"],
    cmap="inferno",
    alpha=0.5,
)
axes[3].set_xlabel("UMAP_1")
axes[3].set_ylabel("UMAP_2")
axes[3].set_title("Mitosis: skimage_orientation")

In [ ]:
import seaborn as sns
import pandas as pd
# Create a DataFrame with the mitosis colors and corresponding skimage areas
data = pd.DataFrame({
    "mitosis_color": mitosis_colors,
    "skimage_area": sc_feature_table["skimage_area"]
})

# Draw a boxplot for each mitosis color
sns.boxplot(x="mitosis_color", y="skimage_area", data=data)
plt.xlabel("mitosis: relative time")
plt.ylabel("area")

In [ ]:
sc_feature_table.columns

In [ ]:
import seaborn as sns
import pandas as pd
# Create a DataFrame with the mitosis colors and corresponding skimage areas

def viz_boxplot(key, mitosis_times, use_abs_vals=False):
    vals = sc_feature_table[key]

    if use_abs_vals:
        vals = vals.abs()
    data = pd.DataFrame({
        "mitosis: relative time": mitosis_times,
        key: vals
    })

    # Draw a boxplot for each mitosis color
    sns.boxplot(x="mitosis: relative time", y=key, data=data)
# viz_boxplot("skimage_area", mitosis_colors)

"""eccentricity: float
Eccentricity of the ellipse that has the same second-moments as the region. 
The eccentricity is the ratio of the focal distance (distance between focal points) over the major axis length. 
The value is in the interval [0, 1). When it is 0, the ellipse becomes a circle."""
# viz_boxplot("skimage_eccentricity", mitosis_colors, use_abs_vals=True)
# viz_boxplot("skimage_axis_minor_length", mitosis_colors)
viz_boxplot("skimage_axis_major_length", mitosis_colors)
# viz_boxplot("skimage_orientation", mitosis_colors)

In [ ]:
import matplotlib.pyplot as plt
# plt.scatter(
#     mitosis_colors,
#     sc_feature_table["skimage_area"],
#     cmap="inferno",
#     alpha=0.5,
# )

plt.boxplot(
    mitosis_colors,
    sc_feature_table["skimage_area"],
    
)

plt.xlabel("Relative time")
plt.ylabel("Area")
plt.title("Mitosis: area vs. relative time")
# add colorbar
plt.colorbar()